In [12]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('slow',12),('fast',5),('sig',3)
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Generating MACD indicator
        # Could also use the inbuilt MACD indicator available in BT framework
        self.sma= bt.indicators.ExponentialMovingAverage(self.data, period=self.params.slow)
        self.fma= bt.indicators.ExponentialMovingAverage(self.data, period=self.params.fast)
        self.macd=self.fma-self.sma
        self.sig= bt.indicators.SimpleMovingAverage(self.macd, period=self.params.sig)
        self.rsi= bt.indicators.RelativeStrengthIndex(self.data, period=3)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        print()

    def next(self):

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            if  self.macd>self.sig: # If MACD indicator crosses its simple moving average
                                    # go bullish on the stock
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if 70 < self.rsi[0]: # Using rsi for exiting around the point when 
                                 # the maxima in macd lines occur so as to maximize profits
                                 # Also keeping a low threshold for rsi so as to trade more frequently although with less profit margins
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()                
                


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)
    
    datapath = ('AAPL.csv')

    # Create a Data Feed
    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        dtformat=('%Y-%m-%d'),
        tmformat=('%H.%M.%S'),
       
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()

Starting Portfolio Value: 100000.00
2019-09-30, BUY CREATE, 55.99
2019-10-01, BUY EXECUTED, Price: 56.27, Cost: 5626.75, Comm 0.00
2019-10-01, SELL CREATE, 56.15
2019-10-02, SELL EXECUTED, Price: 55.76, Cost: 5626.75, Comm 0.00
2019-10-02, OPERATION PROFIT, GROSS -50.25, NET -50.25

2019-10-04, BUY CREATE, 56.75
2019-10-07, BUY EXECUTED, Price: 56.57, Cost: 5656.75, Comm 0.00
2019-10-07, SELL CREATE, 56.76
2019-10-08, SELL EXECUTED, Price: 56.46, Cost: 5656.75, Comm 0.00
2019-10-08, OPERATION PROFIT, GROSS -11.25, NET -11.25

2019-10-08, BUY CREATE, 56.10
2019-10-09, BUY EXECUTED, Price: 56.76, Cost: 5675.75, Comm 0.00
2019-10-10, SELL CREATE, 57.52
2019-10-11, SELL EXECUTED, Price: 58.24, Cost: 5675.75, Comm 0.00
2019-10-11, OPERATION PROFIT, GROSS 148.00, NET 148.00

2019-10-11, BUY CREATE, 59.05
2019-10-14, BUY EXECUTED, Price: 58.72, Cost: 5872.50, Comm 0.00
2019-10-14, SELL CREATE, 58.97
2019-10-15, SELL EXECUTED, Price: 59.10, Cost: 5872.50, Comm 0.00
2019-10-15, OPERATION PROFIT

2020-07-16, BUY EXECUTED, Price: 96.56, Cost: 9656.25, Comm 0.00
2020-07-20, SELL CREATE, 98.36
2020-07-21, SELL EXECUTED, Price: 99.17, Cost: 9656.25, Comm 0.00
2020-07-21, OPERATION PROFIT, GROSS 261.00, NET 261.00

2020-07-29, BUY CREATE, 95.04
2020-07-30, BUY EXECUTED, Price: 94.19, Cost: 9418.75, Comm 0.00
2020-07-31, SELL CREATE, 106.26
2020-08-03, SELL EXECUTED, Price: 108.20, Cost: 9418.75, Comm 0.00
2020-08-03, OPERATION PROFIT, GROSS 1401.25, NET 1401.25

2020-08-03, BUY CREATE, 108.94
2020-08-04, BUY EXECUTED, Price: 109.13, Cost: 10913.25, Comm 0.00
2020-08-04, SELL CREATE, 109.67
2020-08-05, SELL EXECUTED, Price: 109.38, Cost: 10913.25, Comm 0.00
2020-08-05, OPERATION PROFIT, GROSS 24.50, NET 24.50

2020-08-05, BUY CREATE, 110.06
2020-08-06, BUY EXECUTED, Price: 110.40, Cost: 11040.50, Comm 0.00
2020-08-06, SELL CREATE, 113.90
2020-08-07, SELL EXECUTED, Price: 113.21, Cost: 11040.50, Comm 0.00
2020-08-07, OPERATION PROFIT, GROSS 280.00, NET 280.00

2020-08-13, BUY CREATE, 